In [1]:
import requests
from bs4 import BeautifulSoup 
from pathlib import Path
import os
import sys
import time
import re
import traceback
import json

In [2]:
#https://www.gsaelibrary.gsa.gov/ElibMain/contractorList.do?contractorListFor=A

basicLink1="https://www.gsaelibrary.gsa.gov/ElibMain/"
basicLink2 = "contractorList.do?contractorListFor="   #seperate because basicLink1 is used by sub urls too.
pageLetter='A'
downloadPath = "D:\\mystuff\\Junaid Ahmed\\" 
bad_chars = "\\/:?*\"<>|"
bad_chars_list = ["\\","/",":","?","*","\"","<",">","|"]

dataDictionaryList_FilePath = "D:\\mystuff\\Junaid Ahmed\\DataDictionary" 
gsaList_FilePath = "D:\\mystuff\\Junaid Ahmed\\gsaList" 
noTableList_FilePath = "D:\\mystuff\\Junaid Ahmed\\noTableList" 

In [32]:
# Already initialized. so commented
#dataDictionaryList = []
#gsaDictionaryList = []
#noTableOrFileDictionaryList = []

# link = outer link, list wala page hai ye
# subLink = list k har item ka page ka link
# linkCounter 
# fileFinalURL
# linkText

In [4]:
def RemoveUnwwantedCharacters(myStr):
    myStr2 = ""
    for letter in myStr:
        if letter not in bad_chars_list:
            myStr2 += letter
    return myStr2

In [5]:
def GetLink(link):
    try:
        time.sleep(1)
        request=requests.get(link)
    except requests.exceptions.ConnectionError:
        print("Connection refused")
        print("Sleeping for 5 seconds")
        time.sleep(5)
        try:
            request=requests.get(link)
        except requests.exceptions.ConnectionError:
            print("Connection refused again.")
            print("Sleeping for 15 seconds")
            time.sleep(15)
            try:
                request=requests.get(link)
            except requests.exceptions.ConnectionError:
                print("Connection refused again.")
                print("Sleeping for 30 seconds")
                time.sleep(30)
                try:
                    request=requests.get(link)
                except requests.exceptions.ConnectionError:
                    print("Connection refused again.")
                    print("Sleeping for 60 seconds")
                    time.sleep(60)
                    request=requests.get(link)
    return request

In [6]:
def downloadFile(url, fileName):
    with open(fileName, "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [26]:
# link = outer link, list wala page hai ye
# subLink = list k har item ka page ka link
# linkCounter 
# fileFinalURL
# linkText

def UpdateDictionaryList(dictList,dict_var):
    dictList.append(dict_var)                    # append doesnt return anything so it will just  append to refrenced list

def makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML):
    dataDictionary = {}
    dataDictionary["Base Link"] = str(link)
    dataDictionary["Base Link Text"] = str(linkText)
    dataDictionary["Link Number In Base Link"] = str(linkCounter)
    dataDictionary["Specific Contract Information Link "] = str(subLink)
    dataDictionary["isFileExist"] = str(isFileExist)
    dataDictionary["File URL"] = str(fileFinalURL)
    dataDictionary["contractor Information Table HTML"] = str(contractorInformationHTML)
    return dataDictionary
    

In [9]:
def makeFolderIfNotExist(pageLetter,lastFolderName):
    Path(downloadPath+pageLetter+"\\"+folderName).mkdir(parents=True, exist_ok=True,mode=0o777)


In [29]:
def write_record(filePath,record):
    with open(filePath, 'w') as file:
        file.write(json.dumps(record,indent=2)) # use `json.loads` to do the reverse

def append_record(filePath,record):
    with open(filePath, 'a') as f:
        f.write(json.dumps(record, indent=2))
        
def read_record(filePath):
    with open(filePath) as f:
        my_list = [json.loads(line) for line in f]
    return my_list

In [11]:
link=basicLink1 + basicLink2 + pageLetter

totalTime = 0
avgTime = 0

r = GetLink(link)
    
soup=BeautifulSoup(r.text,'html.parser')

In [ ]:


rows = soup.find("table", border=1).find_all("tr")

linkCounter = 0
processedLinkCounter = 0
gsaList = []
try:
    print("Total Number of rows : " + str(len(rows)))
    for row in rows:

        columns = row.find_all("td")
        for column in columns:        

            linkCounter+=1
            print(linkCounter,end=", ")
            if linkCounter<=0:     #links already processed
                continue

            processedLinkCounter+=1
            startTime = time.time()
            linkText = column.get_text()
            folderName = RemoveUnwwantedCharacters (linkText)
            subURL = row.find_all("td")[0].find('a',href=True)['href']

            subLink=basicLink1 + subURL        
            subRequest = GetLink(subLink)

            subSoup=BeautifulSoup(subRequest.text,'html.parser')
            contractorInformationHTML = subSoup.find_all("table")[10]

            try:
                FileTable = subSoup.find_all("table")[8].find_all("table")[7].find_all("tr")[1].find_all("td")[3].find_all("a")
            except exception as e:   #if there is no File table in contractor information
                    isFileExist = False
                    print("-----------------------------------------------------------------------------------------")
                    noTableOrFileDictionaryList.append(linkCounter)
                    print("There is no table..")
                    print("linkCounter : ",linkCounter)
                    print("-----------------------------------------------------------------------------------------")
                    fileFinalURL ="No file avaiable"
                    dataDictionary = makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML)
                    UpdateDictionaryList(noTableOrFileDictionaryList,dataDictionary)
                    makeFolderIfNotExist(pageLetter,folderName)
                    destinationPath = (downloadPath+pageLetter+"\\"+folderName+"\\"+"content.json")
                    write_record(destinationPath,dataDictionary)
                    continue
            if len(FileTable) == 0:    #if there is no File in contractor information
                    isFileExist = False
                    print("-----------------------------------------------------------------------------------------")
                    noTableOrFileDictionaryList.append(linkCounter)
                    print("There is no file..")
                    print("linkCounter : ",linkCounter)
                    print("-----------------------------------------------------------------------------------------")
                    fileFinalURL ="No file avaiable"
                    dataDictionary = makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML)
                    UpdateDictionaryList(noTableOrFileDictionaryList,dataDictionary)                    
                    makeFolderIfNotExist(pageLetter,folderName)
                    destinationPath = (downloadPath+pageLetter+"\\"+folderName+"\\"+"content.json")
                    write_record(destinationPath,dataDictionary)
            else:                      # if there is  atleast one file
                
                for fileTablePart in FileTable:                    # there can be multiple files in one link so traverse over all of them
                    FileHtmURL = fileTablePart["href"]
                    fileUrlShort = FileHtmURL.rsplit("/",1)[0]+"/"
                    if (FileHtmURL == "http://www.gsa.gov/s2"):    # some file links are gsa.gov instead of htm links (htm links redirects to either pdf or docx file)
                        isFileExist = False
                        print("-----------------------------------------------------------------------------------------")
                        gsaDictionaryList.append(linkCounter)
                        print("File link is = http://www.gsa.gov/s2")
                        print("linkCounter : ",linkCounter)
                        print("-----------------------------------------------------------------------------------------")
                        fileFinalURL ="http://www.gsa.gov/s2"
                        dataDictionary = makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML)
                        UpdateDictionaryList(gsaDictionaryList,dataDictionary)                    
                        makeFolderIfNotExist(pageLetter,folderName)
                        destinationPath = (downloadPath+pageLetter+"\\"+folderName+"\\"+"content.json")
                        write_record(destinationPath,dataDictionary)
                    else:      # file exists and has htm url and that means it will have a file. need to go to htm url and meta tag for actual file url
                        isFileExist = True
                        fileFinalURL = FileHtmURL
                        dataDictionary = makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML)
                        UpdateDictionaryList(dataDictionaryList,dataDictionary)                    
                        makeFolderIfNotExist(pageLetter,folderName)
                        destinationPath = (downloadPath+pageLetter+"\\"+folderName+"\\"+"content.json")
                        write_record(destinationPath,dataDictionary)

            endTime = time.time()
            totalTime = totalTime + (endTime-startTime)
            avgTime = totalTime/processedLinkCounter

            if linkCounter%10==0:
                print("\n Total links  = "+ str(linkCounter))
                print("Total links processed = "+ str(processedLinkCounter))
                print("Total elapsed time (in seconds) = "+str(totalTime))
                print("Average link time (in seconds)  = "+ str(avgTime))
                print("\n ")
except Exception as e:
    print("\n\n General error occured. ");
    print("last instances of below items are : ")
    print("linkCounter = " + str(linkCounter))
    print("pageLetter = " + str(pageLetter))
    print("link = " + str(link))
    print("linkText = " + str(linkText))
    print("subLink = " + str(subLink))
    print("fileFinalURL = " + str(fileFinalURL))
    print("folderName = " + str(folderName))
#     print("fileName = " + str(fileName))
    print("destinationPath = " + str(destinationPath))
    print("Error : ")
    print(e)
    print("Traceback: ")
    print(traceback.format_exc())
    
    

Total Number of rows : 571
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
 Total links  = 10
Total links processed = 10
Total elapsed time (in seconds) = 127.11577129364014
Average link time (in seconds)  = 12.711577129364013

 
11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
 Total links  = 20
Total links processed = 20
Total elapsed time (in seconds) = 155.7317180633545
Average link time (in seconds)  = 7.786585903167724

 
21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 
 Total links  = 30
Total links processed = 30
Total elapsed time (in seconds) = 182.83833646774292
Average link time (in seconds)  = 6.0946112155914305

 
31, -----------------------------------------------------------------------------------------
There is no file..
linkCounter :  31
-----------------------------------------------------------------------------------------
32, -----------------------------------------------------------------------------------------
There is no file..
linkCounter :  32
--------------------------------------------

In [54]:
# dataDictionaryList = dataDictionaryList[:-1]

In [ ]:
link=basicLink1 + basicLink2 + pageLetter

totalTime = 0
avgTime = 0

r = GetLink(link)
    
soup=BeautifulSoup(r.text,'html.parser')

rows = soup.find("table", border=1).find_all("tr")

linkCounter = 0
processedLinkCounter = 0
gsaList = []
try:
    print("Total Number of rows : " + str(len(rows)))
    for row in rows:

        columns = row.find_all("td")
        for column in columns:        

            linkCounter+=1
            print(linkCounter,end=", ")
            if linkCounter<=153:     #links already processed
                continue

            processedLinkCounter+=1
            startTime = time.time()
            linkText = column.get_text()
            subURL = row.find_all("td")[0].find('a',href=True)['href']

            subLink=basicLink1 + subURL        
            subRequest = GetLink(subLink)

            subSoup=BeautifulSoup(subRequest.text,'html.parser')
            try:
                FileTable = subSoup.find_all("table")[8].find_all("table")[7].find_all("tr")[1].find_all("td")[3].find_all("a")
            except exception as e:   #if there is no file in contractor information
                    print("-----------------------------------------------------------------------------------------")
                    noTableList.append(linkCounter)
                    print("There is no file table..")
                    print("linkCounter : ",linkCounter)
                    print("-----------------------------------------------------------------------------------------")
                    
                    continue
            #its not logging for pages with no files, so run this again afterwards.
            for fileTablePart in FileTable:
                FileHtmURL = fileTablePart["href"]
                fileUrlShort = FileHtmURL.rsplit("/",1)[0]+"/"
                if (FileHtmURL == "http://www.gsa.gov/s2"):
                    #not added in dictionary
                    print("-----------------------------------------------------------------------------------------")
                    gsaList.append(linkCounter)
                    print("File link is = http://www.gsa.gov/s2 ")
                    print("linkCounter : ",linkCounter)
                    print("-----------------------------------------------------------------------------------------")
                else:
                    FileHtmRequest = GetLink(FileHtmURL)
                    FileHtmSoup =BeautifulSoup(FileHtmRequest.text,'html.parser')
                    FileHtmResult = FileHtmSoup.find("meta")["content"]
                    FilemetaURL = FileHtmSoup.find("meta")["content"].split("url=")[1]
                    fileFinalURL = fileUrlShort+FilemetaURL   

                    UpdateDictionaryList(link,subLink,linkCounter, fileFinalURL, linkText)

                    folderName = RemoveUnwwantedCharacters (linkText) + "\\"
                    Path(downloadPath+folderName).mkdir(parents=True, exist_ok=True,mode=0o777)
                    url = fileFinalURL   
                    extension = fileFinalURL.split(".")[-1].lower()
                    fileName = FileHtmURL.rsplit("/",2)[1] + "." +extension
                    fileName = RemoveUnwwantedCharacters(fileName)
                    destinationPath = (downloadPath+pageLetter+"\\"+folderName+fileName)
                    downloadFile(url, destinationPath)

            endTime = time.time()
            totalTime = totalTime + (endTime-startTime)
            avgTime = totalTime/processedLinkCounter

            if linkCounter%10==0:
                print("\n\n Total links  = "+ str(linkCounter))
                print("Total links processed = "+ str(processedLinkCounter))
                print("Total elapsed time (in seconds) = "+str(totalTime))
                print("Average link time (in seconds)  = "+ str(avgTime))
                print("\n\n ")
except Exception as e:
    print("\n\n General error occured. ");
    print("last instances of below items are : ")
    print("linkCounter = " + str(linkCounter))
    print("pageLetter = " + str(pageLetter))
    print("link = " + str(link))
    print("linkText = " + str(linkText))
    print("subLink = " + str(subLink))
    print("fileFinalURL = " + str(fileFinalURL))
    print("folderName = " + str(folderName))
    print("fileName = " + str(fileName))
    print("destinationPath = " + str(destinationPath))
    print("Error : ")
    print(e)
    print("Traceback: ")
    print(traceback.format_exc())
    
    

In [43]:
# These pages are unique and have no table
# ---------------------------------------
#linkCounter = 151
#pageLetter = B
#sublink = https://www.gsaelibrary.gsa.gov/ElibMain/SearchResults;jsessionid=SQEyLT4aQw7OriGhJJunmNZo.prd2pweb64?searchType=exactWords&coSearch=Y&searchText=BALL+AEROSPACE+%26+TECHNOLOGIES+CORP.
# ---------------------------------------
#linkCounter = 152
#pageLetter = B
#sublink = https://www.gsaelibrary.gsa.gov/ElibMain/SearchResults;jsessionid=eKdbFPm3lrkurrKCqjGxtzr4.prd2pweb64?searchType=exactWords&coSearch=Y&searchText=BALL+AEROSPACE+%26+TECHNOLOGIES+CORP.


